<h1 align=center><font size = 5>Investigation Neighborhoods in Ibaraki-prefecture, Japan for openning a Coffee Shop</font></h1>

## 1. Scrape Population data from public sites using Beautifulsoup

In [1]:
# importing the libraries
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import csv
import re
from urllib.request import urlopen

# Step 1: Sending a HTTP request to a URL
url = "https://www.pref.ibaraki.jp/kikaku/tokei/fukyu/tokei/betsu/jinko/getsu/jinko2001.html"
# Make a GET request to fetch the raw HTML content
r = requests.get(url)
content_type_encoding = r.encoding if r.encoding != 'ISO-8859-1' else None

In [2]:
# Step 2: Parse the html content
soup = BeautifulSoup(r.content, 'html.parser', from_encoding=content_type_encoding)
#print(soup.prettify()) # print the parsed data of html

In [3]:
tables = soup.find_all('table', class_='datatable')

## 1.1. Scrape Table_1

In [4]:
#SCRAPE TABLE_1:"表-1.人口と世帯の推移" <=> "PUPULATION AND HOUSEHOLD TRENDS" into a PANDAS DATAFRAME called df0
#get headers & clean the unneccessary characters:
headers_0=tables[0].findAll('th')

for i, head in enumerate(headers_0):
    headers_0[i]=str(headers_0[i]).replace("<th rowspan=\"3\" scope=\"row\">","").replace("<th nowrap=\"nowrap\" rowspan=\"3\" scope=\"row\">","").replace("<th rowspan=\"2\" scope=\"row\">","")
for i, head in enumerate(headers_0):
    headers_0[i]=str(headers_0[i]).replace("</th>","").replace("<br/>\n","").replace("<th nowrap=\"nowrap\" rowspan=\"2\" scope=\"row\">\n<p>","").replace("</p>\n<p>","").replace("</p>\n","")
headers_0.remove('<th colspan=\"6\" scope=\"row\">人口')

#Find all rows data in TABLE_1: 
table0_rows = tables[0].find_all('tr')

#Because the first row is the header, only select data from 2nd row
table0_rows = table0_rows[3:len(table0_rows)]

#Create a list of data in each row
l0 = []
for tr in table0_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l0.append(row)

#Create dataframe & clean the unneccessary characters:
df0 = pd.DataFrame(l0, columns=headers_0)
df0 = df0.replace('\n','', regex=True)
df0 = df0.replace(',','', regex=True)

#Change the first columns (meaning Date) to dataframe index
df0.set_index('年月日', inplace=True)

#Columns & Index were Japanese, I changed all of them to English for readers convenience.
df0newcolumns = ['Total_Household', 'Ave_People_in_Household', 'Total_Population', 'Man', 'Women', 'Population_compared_to_previous_Investigation', 'Percentage_compared_to_previous_Investigation', 'Density_(People/kmsquare)']
df0.columns = df0newcolumns
df0newindex = ['1955_Oct', '1960-Oct', '1965_Oct', '1970_Oct', '1975_Oct', '1980_Oct', '1985_Oct', '1990_Oct', '1995_Oct', '2000_Oct',
               '2005_Oct', '2010_Oct', '2015_Oct', '2018_Dec', '2020_Jan']
df0.index = df0newindex

#Convert all columns of DataFrame to numeric types
df0 = df0.apply(pd.to_numeric)
print('TABLE 1: PUPULATION AND HOUSEHOLD TRENDS')
df0

TABLE 1: PUPULATION AND HOUSEHOLD TRENDS


,Total_Household,Ave_People_in_Household,Total_Population,Man,Women,Population_compared_to_previous_Investigation,Percentage_compared_to_previous_Investigation,Density_(People/kmsquare)
1955_Oct,382315,5.39,2064037,1006093,1057944,24619,1.2,338.9
1960-Oct,409465,5.03,2047024,1000184,1046840,-17013,-0.8,336.2
1965_Oct,447871,4.55,2056154,1007852,1048302,9130,0.4,337.7
1970_Oct,508537,4.16,2143551,1054003,1089548,87397,4.3,352.1
1975_Oct,590131,3.92,2342198,1159707,1182491,198647,9.3,384.6
1980_Oct,692855,3.66,2558007,1272533,1285474,215809,9.2,419.9
1985_Oct,758085,3.56,2725005,1357963,1367042,166998,6.5,447.1
1990_Oct,833634,3.39,2845382,1419117,1426265,120377,4.4,467.0
1995_Oct,922745,3.17,2955530,1476437,1479093,110148,3.9,485.0
2000_Oct,985829,2.99,2985676,1488340,1497336,30146,1.0,489.8


In [5]:
df0.dtypes

Total_Household                                    int64
Ave_People_in_Household                          float64
Total_Population                                   int64
Man                                                int64
Women                                              int64
Population_compared_to_previous_Investigation      int64
Percentage_compared_to_previous_Investigation    float64
Density_(People/kmsquare)                        float64
dtype: object

## 1.2. Scrape Table_2

In [6]:
#SCRAPE TABLE_2: "県の世帯数,人口及び人口移動" <=> "PREFECTURE HOUSEHOLDS, POPULATION, MIGRATION" into a PANDAS DATAFRAME called df1
#get headers & clean the unneccessary characters:
headers_1=tables[1].findAll('th')

for i, head in enumerate(headers_1):
    headers_1[i]=str(headers_1[i]).replace("<th colspan=\"2\" rowspan=\"4\" scope=\"row\">","").replace("<th rowspan=\"4\" scope=\"row\">","").replace("<th colspan=\"3\" rowspan=\"2\" scope=\"row\">","")
for i, head in enumerate(headers_1):
    headers_1[i]=str(headers_1[i]).replace("<th colspan=\"7\" scope=\"row\">","").replace("<th rowspan=\"3\" scope=\"row\">","").replace("<th colspan=\"3\" scope=\"row\">","").replace("</p>\n<p>","").replace("<th rowspan=\"2\" scope=\"row\">","")
for i, head in enumerate(headers_1):
    headers_1[i]=str(headers_1[i]).replace("</th>","").replace("<br/>\n","")

headers_1.remove('人口')
headers_1.remove('前月中の人口移動')
headers_1.remove('自然動態')
headers_1.remove('社会動態')

#Find all rows data in TABLE_2: 
table1_rows = tables[1].find_all('tr')

#Because the first 4 rows are the header, only select data from 2nd row
table1_rows = table1_rows[4:len(table1_rows)]

#Create a list of data in each row
l1 = []
for tr in table1_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l1.append(row)

#Create dataframe & clean the unneccessary characters:
df1 = pd.DataFrame(l1, columns=headers_1)
df1 = df1.replace('\n','', regex=True)
df1 = df1.replace(',','', regex=True)

#Change the first columns (meaning Date) to dataframe index
df1.set_index('年月日', inplace=True)

#Columns & Index were Japanese, I changed all of them to English for readers convenience.
df1newcolumns = ['Total_Household', 'Total_Population', 'Man', 'Women', 'Increase_or_Decrease', 'Natural_Reasons', 'Birth', 'Dead', 'Migration_Reasons', 'In', 'Out']
df1.columns = df1newcolumns
df1newindex = ['2019_Jan', '2019_Feb', '2019_Mar', '2019_Apr', '2019_May', '2019_Jun', '2019_Jul', '2019_Aug', '2019_Sep', '2019_Oct',
               '2019_Nov', '2019_Dec', '2020_Jan']
df1.index = df1newindex

#Convert all columns of DataFrame to numeric types
df1 = df1.apply(pd.to_numeric)
#print('TABLE 2: PREFECTURE HOUSEHOLDS, POPULATION, MIGRATION')
df1

,Total_Household,Total_Population,Man,Women,Increase_or_Decrease,Natural_Reasons,Birth,Dead,Migration_Reasons,In,Out
2019_Jan,1163338,2881020,1438291,1442729,-1120,-1276,1553,2829,156,8057,7901
2019_Feb,1163573,2879225,1437482,1441743,-1795,-2058,1667,3725,263,8297,8034
2019_Mar,1163464,2877206,1436506,1440700,-2019,-1576,1446,3022,-443,8451,8894
2019_Apr,1164984,2870883,1433317,1437566,-6323,-1293,1410,2703,-5030,18524,23554
2019_May,1168885,2871872,1434181,1437691,989,-1200,1426,2626,2189,15840,13651
2019_Jun,1170033,2871199,1433974,1437225,-673,-1330,1691,3021,657,9623,8966
2019_Jul,1170582,2870018,1433374,1436644,-1181,-811,1443,2254,-370,7904,8274
2019_Aug,1171498,2869312,1433170,1436142,-706,-1071,1587,2658,365,9314,8949
2019_Sep,1172116,2868148,1432670,1435478,-1164,-1043,1597,2640,-121,8694,8815
2019_Oct,1173307,2868041,1432563,1435478,-107,-1052,1586,2638,945,9332,8387


In [7]:
df1.dtypes

Total_Household         int64
Total_Population        int64
Man                     int64
Women                   int64
Increase_or_Decrease    int64
Natural_Reasons         int64
Birth                   int64
Dead                    int64
Migration_Reasons       int64
In                      int64
Out                     int64
dtype: object

## 1.3. Scrape Table_3

In [8]:
#SCRAPE TABLE_3: "地域の世帯数,人口及び人口移動" <=> "IBARAKI'S AREAS HOUSEHOLDS, POPULATION, MIGRATION" into a PANDAS DATAFRAME called df2
#get headers & clean the unneccessary characters:
headers_2=tables[2].findAll('th')

for i, head in enumerate(headers_2):
    headers_2[i]=str(headers_2[i]).replace("<th colspan=\"2\" rowspan=\"4\" scope=\"row\">","").replace("<th rowspan=\"4\" scope=\"row\">","").replace("<th colspan=\"3\" rowspan=\"2\" scope=\"row\">","")
for i, head in enumerate(headers_2):
    headers_2[i]=str(headers_2[i]).replace("<th colspan=\"7\" scope=\"row\">","").replace("<th rowspan=\"3\" scope=\"row\">","").replace("<th colspan=\"3\" scope=\"row\">","").replace("</p>\n<p>","").replace("<th rowspan=\"2\" scope=\"row\">","")
for i, head in enumerate(headers_2):
    headers_2[i]=str(headers_2[i]).replace("</th>","").replace("<br/>\n","").replace("<th nowrap=\"nowrap\" rowspan=\"2\" scope=\"row\">","")

headers_2.remove('人口')
headers_2.remove('前月中の人口移動')
headers_2.remove('自然動態')
headers_2.remove('社会動態')

#Find all rows data in TABLE_3: 
table2_rows = tables[2].find_all('tr')

#Because the first 4 rows are the header, only select data from 2nd row
table2_rows = table2_rows[4:len(table2_rows)]

#Create a list of data in each row
l2 = []
for tr in table2_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l2.append(row)

#Create dataframe & clean the unneccessary characters:
df2 = pd.DataFrame(l2, columns=headers_2)
df2 = df2.replace('\n','', regex=True)
df2 = df2.replace(',','', regex=True)

#Change the first columns (meaning Date) to dataframe index
df2.set_index('地域', inplace=True)

#Columns & Index were Japanese, I changed all of them to English for readers convenience.
df2newcolumns = ['Total_Household', 'Total_Population', 'Man', 'Women', 'Increase_or_Decrease', 'Natural_Reasons', 'Birth', 'Dead', 'Migration_Reasons', 'In', 'Out']
df2.columns = df2newcolumns
df2newindex = ['City_Area', 'District_Area', 'North_Prefecture', 'Middle_Prefecture', 'Rooko_Area', 'South_Prefecture', 'West_Prefecture']
df2.index = df2newindex

#Convert all columns of DataFrame to numeric types
df2 = df2.apply(pd.to_numeric)
print('TABLE 3: IBARAKI\'S AREAS HOUSEHOLDS, POPULATION, MIGRATION')
df2


TABLE 3: IBARAKI'S AREAS HOUSEHOLDS, POPULATION, MIGRATION


,Total_Household,Total_Population,Man,Women,Increase_or_Decrease,Natural_Reasons,Birth,Dead,Migration_Reasons,In,Out
City_Area,1074498,2606801,1302010,1304791,-773,-1118,1399,2517,345,7509,7164
District_Area,101396,259524,129715,129809,-169,-204,110,314,35,720,685
North_Prefecture,248615,595827,296583,299244,-423,-380,275,655,-43,1135,1178
Middle_Prefecture,195891,459301,225952,233349,-147,-202,244,446,55,1055,1000
Rooko_Area,109606,269474,137977,131497,-52,-141,137,278,89,803,714
South_Prefecture,421101,1004639,501743,502896,-2,-302,583,885,300,3587,3287
West_Prefecture,200681,537084,269470,267614,-318,-297,270,567,-21,1649,1670


In [9]:
df2.dtypes

Total_Household         int64
Total_Population        int64
Man                     int64
Women                   int64
Increase_or_Decrease    int64
Natural_Reasons         int64
Birth                   int64
Dead                    int64
Migration_Reasons       int64
In                      int64
Out                     int64
dtype: object

## 1.4. Scrape Table_4-1

In [10]:
#SCRAPE TABLE_4-1: "市町村の世帯数,人口及び人口移動" <=> "CITIES, TOWNS & VILLAGES HOUSEHOLDS, POPULATION, MIGRATION" into a PANDAS DATAFRAME called df3
#get headers & clean the unneccessary characters:
headers_3=tables[3].findAll('th')

for i, head in enumerate(headers_3):
    headers_3[i]=str(headers_3[i]).replace("<th colspan=\"2\" rowspan=\"4\" scope=\"row\">","").replace("<th rowspan=\"4\" scope=\"row\">","").replace("<th colspan=\"3\" rowspan=\"2\" scope=\"row\">","")
for i, head in enumerate(headers_2):
    headers_3[i]=str(headers_3[i]).replace("<th colspan=\"7\" scope=\"row\">","").replace("<th rowspan=\"3\" scope=\"row\">","").replace("<th colspan=\"3\" scope=\"row\">","").replace("</p>\n<p>","").replace("<th rowspan=\"2\" scope=\"row\">","")
for i, head in enumerate(headers_3):
    headers_3[i]=str(headers_3[i]).replace("</th>","").replace("<br/>\n","").replace("<th nowrap=\"nowrap\" rowspan=\"2\" scope=\"row\">","")

headers_3.remove('人口')
headers_3.remove('前月中の人口移動')
headers_3.remove('自然動態')
headers_3.remove('社会動態')

#Find all rows data in TABLE_4-1: 
table3_rows = tables[3].find_all('tr')

#Because the first 4 rows are the header, only select data from 2nd row
table3_rows = table3_rows[4:len(table3_rows)]

#Create a list of data in each row
l3 = []
for tr in table3_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l3.append(row)

#Create dataframe & clean the unneccessary characters:
df3 = pd.DataFrame(l3, columns=headers_3)
df3 = df3.replace('\n','', regex=True)
df3 = df3.replace(',','', regex=True)

#Change the first columns (meaning Date) to dataframe index
df3.set_index('市町村', inplace=True)

#Columns & Index were Japanese, I changed all of them to English for readers convenience.
df3newcolumns = ['Total_Household', 'Total_Population', 'Man', 'Women', 'Increase_or_Decrease', 'Natural_Reasons', 'Birth', 'Dead', 'Migration_Reasons', 'In', 'Out']
df3.columns = df3newcolumns
df3newindex = ['Mito-shi', 'Hitachi-shi', 'Tsuchiura-shi', 'Koga-shi', 'Ishioka-shi', 'Yuki-shi', 'Ryugasaki-shi', 'Shimozuma-shi', 'Joso-shi', 'HitachiOta-shi',
                'Takahagi-shi', 'KitaIbaraki-shi', 'Sagama-shi', 'Toride-shi', 'Ushiku-shi', 'Tsukuba-shi', 'Hitachinaka-shi', 'Kashima-shi', 'Itako-shi', 'Moriya-shi',
                'HitachiOmiya-shi', 'Naka-shi', 'Chikusei-shi', 'Bando-shi', 'Inashiki-shi', 'Kasumigaura-shi', 'Sakuragawa-shi', 'Kamisu-shi', 'Namegata-shi', 'Hokota-shi',
                'Tsukubamirai-shi', 'Omitama-shi']
df3.index = df3newindex

#Convert all columns of DataFrame to numeric types
df3 = df3.apply(pd.to_numeric)
print('TABLE 4-1: CITIES, TOWNS & VILLAGES AREAS HOUSEHOLDS, POPULATION, MIGRATION')
df3

TABLE 4-1: CITIES, TOWNS & VILLAGES AREAS HOUSEHOLDS, POPULATION, MIGRATION


,Total_Household,Total_Population,Man,Women,Increase_or_Decrease,Natural_Reasons,Birth,Dead,Migration_Reasons,In,Out
Mito-shi,122398,269763,132189,137574,-51,-68,155,223,17,593,576
Hitachi-shi,78163,175635,87549,88086,-158,-96,70,166,-62,284,346
Tsuchiura-shi,60369,138557,69126,69431,14,-60,79,139,74,480,406
Koga-shi,55670,139107,69691,69416,-46,-63,75,138,17,495,478
Ishioka-shi,28107,72968,36121,36847,-30,-49,37,86,19,185,166
Yuki-shi,19253,50748,25460,25288,-62,-40,20,60,-22,143,165
Ryugasaki-shi,31956,76723,38243,38480,-50,-20,41,61,-30,216,246
Shimozuma-shi,15832,41861,21013,20848,-54,-21,22,43,-33,123,156
Joso-shi,21826,59675,29875,29800,-3,-22,28,50,19,287,268
HitachiOta-shi,19307,48605,23679,24926,-42,-51,18,69,9,82,73


In [11]:
df3.dtypes

Total_Household         int64
Total_Population        int64
Man                     int64
Women                   int64
Increase_or_Decrease    int64
Natural_Reasons         int64
Birth                   int64
Dead                    int64
Migration_Reasons       int64
In                      int64
Out                     int64
dtype: object

## 1.5. Scrape Table_4-2

In [12]:
#SCRAPE TABLE_4-2: "市町村の世帯数,人口及び人口移動" <=> "CITIES, TOWNS & VILLAGES HOUSEHOLDS, POPULATION, MIGRATION" into a PANDAS DATAFRAME called df4
#get headers & clean the unneccessary characters:
headers_4=tables[4].findAll('th')

for i, head in enumerate(headers_4):
    headers_4[i]=str(headers_4[i]).replace("<th colspan=\"2\" rowspan=\"4\" scope=\"row\">","").replace("<th rowspan=\"4\" scope=\"row\">","").replace("<th colspan=\"3\" rowspan=\"2\" scope=\"row\">","")
for i, head in enumerate(headers_4):
    headers_4[i]=str(headers_4[i]).replace("<th colspan=\"7\" scope=\"row\">","").replace("<th rowspan=\"3\" scope=\"row\">","").replace("<th colspan=\"3\" scope=\"row\">","").replace("</p>\n<p>","").replace("<th rowspan=\"2\" scope=\"row\">","")
for i, head in enumerate(headers_4):
    headers_4[i]=str(headers_4[i]).replace("</th>","").replace("<br/>\n","").replace("<th nowrap=\"nowrap\" rowspan=\"2\" scope=\"row\">","")

headers_4.remove('人口')
headers_4.remove('前月中の人口移動')
headers_4.remove('自然動態')
headers_4.remove('社会動態')

#Find all rows data in TABLE_4-2
table4_rows = tables[4].find_all('tr')

#Because the first 4 rows are the header, only select data from 2nd row
table4_rows = table4_rows[4:len(table4_rows)]

#Create a list of data in each row
l4 = []
for tr in table4_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l4.append(row)

#Create dataframe & clean the unneccessary characters:
df4 = pd.DataFrame(l4, columns=headers_4)
df4 = df4.replace('\n','', regex=True)
df4 = df4.replace(',','', regex=True)

#Change the first columns (meaning Date) to dataframe index
df4.set_index('市町村', inplace=True)

#Columns & Index were Japanese, I changed all of them to English for readers convenience.
df4newcolumns = ['Total_Household', 'Total_Population', 'Man', 'Women', 'Increase_or_Decrease', 'Natural_Reasons', 'Birth', 'Dead', 'Migration_Reasons', 'In', 'Out']
df4.columns = df3newcolumns
df4newindex = ['HigashiIbaraki-gun', 'Ibaraki-cho', 'Oarai-cho', 'Shirosato-cho', 'Naka-gun', 'Tokai-mura', 'Kuji-gun', 'Daigo-cho', 'Inashiki-gun', 'Miho-mura',
                'Ami-cho', 'Kawachi-cho', 'Yuki-gun', 'Yachio-cho', 'Sashima-gun', 'Goka-cho', 'Sakai-cho', 'Kitasoma-gun', 'Tone-cho']
df4.index = df4newindex

#Convert all columns of DataFrame to numeric types
df4 = df4.apply(pd.to_numeric)
print('TABLE 4-2: CITIES, TOWNS & VILLAGES AREAS HOUSEHOLDS, POPULATION, MIGRATION')
df4

TABLE 4-2: CITIES, TOWNS & VILLAGES AREAS HOUSEHOLDS, POPULATION, MIGRATION


,Total_Household,Total_Population,Man,Women,Increase_or_Decrease,Natural_Reasons,Birth,Dead,Migration_Reasons,In,Out
HigashiIbaraki-gun,26048,66124,32615,33509,-46,-72,22,94,26,157,131
Ibaraki-cho,11971,31785,15728,16057,-40,-31,11,42,-9,63,72
Oarai-cho,6842,15987,7930,8057,4,-21,7,28,25,62,37
Shirosato-cho,7235,18352,8957,9395,-10,-20,4,24,10,32,22
Naka-gun,14985,37688,19029,18659,-8,1,29,28,-9,89,98
Tokai-mura,14985,37688,19029,18659,-8,1,29,28,-9,89,98
Kuji-gun,6450,15993,7802,8191,-63,-41,1,42,-22,17,39
Daigo-cho,6450,15993,7802,8191,-63,-41,1,42,-22,17,39
Inashiki-gun,28809,70809,35208,35601,13,-33,33,66,46,240,194
Miho-mura,5854,14593,7417,7176,11,-10,5,15,21,61,40


In [25]:
df4.dtypes

Total_Household         int64
Total_Population        int64
Man                     int64
Women                   int64
Increase_or_Decrease    int64
Natural_Reasons         int64
Birth                   int64
Dead                    int64
Migration_Reasons       int64
In                      int64
Out                     int64
dtype: object

## 1.6. Scrape Table_5

In [24]:
#SCRAPE TABLE_5: 人口順位 - POPULATION RANKS into a PANDAS DATAFRAME called df5
#get headers & clean the unneccessary characters:
headers_5=tables[5].findAll('th')
for i, head in enumerate(headers_5):
    headers_5[i]=str(headers_5[i]).replace("<th scope=\"row\">","").replace("<br/>","").replace("</th>","").replace("\n","")

#Find all rows data in TABLE_5
table5_rows = tables[5].find_all('tr')

#Because the first row is the header, only select data from 2nd row
table5_rows = table5_rows[1:len(table5_rows)]

#Create a list of data in each row
l5 = []
for tr in table5_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l5.append(row)

#Create dataframe & clean the unneccessary characters:
df5 = pd.DataFrame(l5, columns=headers_5)
df5 = df5.replace('\n','', regex=True)
df5 = df5.replace(',','', regex=True)

#Change the first columns (meaning Date) to dataframe index
df5.set_index('市町村', inplace=True)

#Columns & Index were Japanese, I changed all of them to English for readers convenience.
df5newcolumns = ['Rank', 'Total_Population']
df5.columns = df5newcolumns
df5newindex = ['Mito-shi','Tsukuba-shi','Hitachi-shi','Hitachinaka-shi','Koga-shi','Tsuchiura-shi','Toride-shi','Chikusei-shi','Kamisu-shi','Ushiku-shi',
               'Ryugasaki-shi','Sagama-shi','Ishioka-shi','Moriya-shi','Kashima-shi','Joso-shi','Naka-shi','Bando-shi','Tsukubamirai-shi','Yuki-shi',
               'Omitama-shi','HitachiOta-shi','Ami-cho','Hokota-shi','KitaIbaraki-shi','Shimozuma-shi','Kasumigaura-shi','HitachiOmiya-shi','Inashiki-shi','Sakuragawa-shi',
               'Tokai-mura','Namegata-shi','Ibaraki-cho','Takahagi-shi','Itako-shi','Sakai-cho','Yachio-cho','Shirosato-cho','Daigo-cho','Oarai-cho',
               'Tone-cho','Miho-mura','Kawachi-cho','Goka-cho']
df5.index = df5newindex

#Convert all columns of DataFrame to numeric types
df5 = df5.apply(pd.to_numeric)
print('TABLE 5: POPULATION RANKS')
df5

TABLE 5: POPULATION RANKS


,Rank,Total_Population
Mito-shi,1,269763
Tsukuba-shi,2,241808
Hitachi-shi,3,175635
Hitachinaka-shi,4,155045
Koga-shi,5,139107
Tsuchiura-shi,6,138557
Toride-shi,7,104611
Chikusei-shi,8,100658
Kamisu-shi,9,95428
Ushiku-shi,10,84589


In [26]:
df5.dtypes

Rank                int64
Total_Population    int64
dtype: object

## 1.7. Scrape Table_6

In [30]:
#SCRAPE TABLE_6: "表-4.最近1年間の人口増減数" <=> Changes in population over the last year into a PANDAS DATAFRAME called df7
#get headers & clean the unneccessary characters:
headers_6=tables[6].findAll('th')
for i, head in enumerate(headers_6):
    headers_6[i]=str(headers_6[i]).replace("<th scope=\"row\">","").replace("<br/>","").replace("</th>","").replace("\n","")

#Find all rows data in 7th table: "表-4.最近1年間の人口増減数" <=> Changes in population over the last year
table6_rows = tables[6].find_all('tr')

#Because the first row is the header, only select data from 2nd row
table6_rows = table6_rows[1:len(table6_rows)]

#Create a list of data in each row
l6 = []
for tr in table6_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l6.append(row)

#Create dataframe & clean the unneccessary characters:
df6 = pd.DataFrame(l6, columns=headers_6)
df6 = df6.replace('\n','', regex=True)
df6 = df6.replace(',','', regex=True)

#Change the first columns (name of the cities in Ibaraki-prefecture) to dataframe index
df6.set_index('市町村', inplace=True)

#Columns & Index were Japanese, I changed all of them to English for readers convenience.
df6newcolumns = ['2018_Dec', '2019_Jan', '2019_Feb', '2019-Mar', '2019-Apr', '2019-May', '2019-Jun', '2019-Jul', '2019-Aug', '2019-Sep', '2019-Oct', '2019-Nov', '2019-Dec', 'Total2019']
df6.columns = df6newcolumns
df6newindex = ['Tsukuba-shi', 'Moriya-shi', 'Ami-cho', 'Tsukubamirai-shi', 'Kamisu-shi', 'Tokai-mura', 'Goka-cho', 'Sakai-cho', 'Ushiku-shi', 'Oarai-cho',
                'Kawachi-cho', 'Yachio-cho', 'Kashima-shi', 'Miho-mura', 'Tone-cho', 'Naka-shi', 'Ibaraki-cho', 'Yuki-shi', 'Ryugasaki-shi', 'Shirosato-cho',
                'Toride-shi', 'Shimozuma-shi', 'Hokota-shi', 'Itako-shi', 'Takahagi-shi', 'Omitama-shi', 'Kasumigaura-shi', 'KitaIbaraki-shi', 'Bando-shi', 'Joso-shi',
                'Mito-shi', 'Daigo-cho', 'Hitachinaka-shi', 'Namegata-shi', 'Sakuragawa-shi', 'Sagama-shi', 'Koga-shi', 'Ishioka-shi', 'HitachiOmiya-shi', 'Inashiki-shi',
                'Tsuchiura-shi', 'HitachiOta-shi', 'Chikusei-shi', 'Hitachi-shi', 'Total']
df6.index = df6newindex

#Convert all columns of DataFrame to numeric types
df6 = df6.apply(pd.to_numeric)

print('TABLE 6: CHANGES IN POPULATION OVER THE LAST YEAR')
df6

TABLE 6: CHANGES IN POPULATION OVER THE LAST YEAR


,2018_Dec,2019_Jan,2019_Feb,2019-Mar,2019-Apr,2019-May,2019-Jun,2019-Jul,2019-Aug,2019-Sep,2019-Oct,2019-Nov,2019-Dec,Total2019
Tsukuba-shi,213,253,102,-303,1384,350,174,215,138,713,437,174,210,3847
Moriya-shi,101,-2,86,11,137,89,6,90,72,80,129,112,54,864
Ami-cho,-19,-20,51,-26,103,-12,-12,41,37,17,44,-6,10,227
Tsukubamirai-shi,-5,27,17,-12,68,75,12,9,8,-9,0,25,-15,205
Kamisu-shi,-38,-26,-13,-133,96,103,21,50,-10,-3,-10,31,37,143
Tokai-mura,24,23,-10,-84,5,13,39,45,-27,16,13,-19,-8,6
Goka-cho,9,-1,1,-32,-14,-5,-8,12,-5,2,-16,-25,-10,-101
Sakai-cho,-38,-27,-6,-49,2,-32,-7,-17,8,-29,42,-3,-14,-132
Ushiku-shi,-25,-3,27,-93,70,-47,-12,-29,-8,-33,33,-35,-46,-176
Oarai-cho,-19,-34,-43,-6,-10,5,-24,-23,-31,-30,-10,1,4,-201


In [31]:
df6.dtypes

2018_Dec     int64
2019_Jan     int64
2019_Feb     int64
2019-Mar     int64
2019-Apr     int64
2019-May     int64
2019-Jun     int64
2019-Jul     int64
2019-Aug     int64
2019-Sep     int64
2019-Oct     int64
2019-Nov     int64
2019-Dec     int64
Total2019    int64
dtype: object

## 2. Analytics & Visualization Population data of Ibaraki

### To find the best place (city) to setup a Coffee shop

## 2.1. Visualiztion the fist table